In [ ]:
!pip install -U datasets huggingface-hub
!pip install googletrans==3.1.0a0

In [ ]:
from datasets import load_dataset
belle = load_dataset(
    "QingyiSi/Alpaca-CoT", 
    data_files="belle_cn/school_math_0.25M.json",
#     data_files="belle_cn/multiturn_chat_0.8M.json",
    split="train",
)

In [ ]:
belle.to_json("/kaggle/working/belle_school_math.jsonl")

In [ ]:
from copy import deepcopy
from googletrans import Translator
translator = Translator()

def translate(example):
    input_e, output_e = example["instruction"], example["output"]
    translations = translator.translate([input_e, output_e], src="zh-CN", dest="vi")
    input_v, output_v = [x.text for x in translations]
    new_example = deepcopy(example)
    new_example["instruction"] = input_v
    new_example["output"] = output_v
    return new_example

In [ ]:
sub_belle = belle.to_list()[:20000]

In [ ]:
from tqdm import tqdm
import json
from datasets import Dataset
savefile = "/kaggle/working/belle_school_math_vi.jsonl"
try:
    continue_from = sum([1 for _ in open(savefile)])
except:
    continue_from = 0

cont_dataset = sub_belle[continue_from:]
print(f"continue translating from sample {continue_from}")

with open(savefile, "a+") as file:
    for example in tqdm(cont_dataset):
        new_example = translate(example)
        d = json.dumps(new_example, ensure_ascii=False) + "\n"
        file.write(d)
#         print(new_example)

In [ ]:
from datasets import load_dataset
belle_vi = load_dataset("json", data_files=savefile, split="train")
belle_vi.push_to_hub()